# Workspace resources

<table align="left">

  <td>
    <a href="https://github.com/DataBiosphere/terra-axon-examples/blob/main/first_hour_on_terra/workspace_resources.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/DataBiosphere/terra-axon-examples/main/first_hour_on_terra/workspace_resources.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in a Terra notebook instance
    </a>
  </td>                                                                                               
</table>

## Overview

This notebook provides examples of working with workspace resources. Build upon the best practices described in this notebook to include and share resources in your own workspaces. 

### Objective

Perform common workspace resource operations including:

1. Viewing the resources available in an existing workspace.
1. Creating new resources in an existing workspace.
1. Viewing data from a cloud resource in your Terra cloud environment.
#### How to run this notebook

Run this notebook cell by cell to set up your workspace. All setup steps are optional, but highly recommended so that your workspace is compatible with the Enterprise Terra tutorials.

#### Costs

This notebook takes less than a minute to run, which will typically cost less than $0.01 of compute time on your cloud environment.

### Notebook setup

#### Set up utility functions

In [ ]:
import json

'''
Resolves bucket URL from bucket reference in workspace.
'''
def get_bucket_url_from_reference(bucket_reference):
    BUCKET_CMD_OUTPUT = !terra resolve --name={bucket_reference}
    BUCKET = BUCKET_CMD_OUTPUT[0]
    return BUCKET

'''
Resolves current workspace ID from workspace description.
'''
CURRENT_WORKSPACE_ID = !terra workspace describe --format=json | jq --raw-output ".id"

#### Workspace setup
<div class="alert alert-block alert-info">
<b>Note:</b> This notebook assumes that <a href="../../terra-axon-examples/workspace_setup.ipynb">`workspace_setup.ipynb`</a> has been run.
</div>
    
`workspace_setup.ipynb` creates two Cloud Storage buckets for your workspace files with workspace reference names: 

 - ws_files   
 - ws_files_autodelete_after_two_weeks      
    
The code in this notebook will write output files to the "autodelete" bucket by default.  
    Any file in this bucket will be automatically deleted <b>two weeks</b> after it is written.  
    This alleviates the need for you to remember to clean up temporary and example files manually.  
    If you want to write outputs to a durable location, simply change the assignment of the `BUCKET_REFERENCE` variable in the cell below and re-run the notebook. 

In [ ]:
# Change this to "ws_files" to use the durable workspace bucket instead of the autodelete bucket.
BUCKET_REFERENCE = "ws_files_autodelete_after_two_weeks"

In [ ]:
MY_BUCKET = get_bucket_url_from_reference(BUCKET_REFERENCE)
print(f'Bucket ID: {MY_BUCKET}')

In [ ]:
WORKSPACE_ID = get_current_workspace_id()
print(f'Workspace ID: {WORKSPACE_ID}')

### Add referenced resources

For each type of referenced resource, this notebook provides
1. A **widget** which runs a Terra CLI command to add a referenced resource.<br>simply fill in the inputs with your desired values and click the button.
1. A **video** showing how to create that type of resource in the Enterprise Terra UI.



In [ ]:
# Widget utilities used in two or more commands
import ipywidgets, os

input_style= {'description_width':'initial'}

cloningDropDown = widgets.Dropdown(
 options=['COPY_NOTHING','COPY_DEFINITION','COPY_RESOURCE','COPY_REFERENCE'],
 value='COPY_REFERENCE',
 description="Cloning:",
 disabled=False,
 )

#### 1. Add a GitHub repository

Both public and private GitHub repositories can be added to data collections. In order to add a private GitHub repository, you must first [set up your Terra SSH key](https://terra-docs.api.verily.com/docs/how_to_guides/terra_ssh_key_guide/https://terra-docs.api.verily.com/docs/how_to_guides/terra_ssh_key_guide/) and should provide a repo URL of the format `git@github.com` instead of `http://`. No additional setup is required to add a public GitHub repository.

Replace the placeholder values in the cell below and run to add a GitHub repository to your data collection. You should see output resembling:
```
Successfully added referenced git repo.
Name:         dc-public-repo
Description:  A public GitHub repo
Type:         GIT_REPO
Stewardship:  REFERENCED
Cloning:      COPY_REFERENCE
Properties:   class Properties {
    []
}
Git repo Url: https://github.com/<YOUR_REPO> Successfully added referenced git repo.
Name:         dc-public-repo
Description:  A public GitHub repo
Type:         GIT_REPO
Stewardship:  REFERENCED
Cloning:      COPY_REFERENCE
Properties:   class Properties {
    []
}
Git repo Url: https://github.com/<YOUR_REPO>
```

In [ ]:
input_name = widgets.Text(
 placeholder="<REPO_NAME>",
 description="Name:"
 )
input_description = widgets.Text(
 placeholder="<REPO_DESCRIPTION>",
 description="Description:"
 )
input_repo_url = widgets.Text(
 placeholder="<REPO_URL>",
 description="URL:"
 )

display(cloningDropDown)
display(input_name)
display(input_description)
display(input_repo_url)

# define a variable that will be reset when the button is pressed
buttonOutput = 'Please provide appropriate values in the input boxes.'

# define a function for the button to call
def button_click_event(b):
    with output:
        global buttonOutput
        
        terraCommand = """terra resource add-ref git-repo \\
        --cloning={0} \\
        --description={1} \\
        --name={2} \\
        --repo-url={3} \\
        --workspace={4}        """.format(cloningDropDown.value,input_description.value,input_name.value,input_repo_url.value,WORKSPACE_ID)       
        print(terraCommand)
        os.system(terraCommand)

# get a reference to the widget output
output = widgets.Output()

button = widgets.Button(
    description='Add resource',
    disabled=False,
    button_style='',
    tooltip='abcd',
    icon='plus'
)

#bind the button_click_event to the button call event
button.on_click(button_click_event)

# show the current state of the output
print(buttonOutput)

#display the button
display(button, output)

#### 2. Add a BigQuery dataset

Replace the placeholder values in the cell below and run to add a GitHub repository to your data collection. You should see output resembling:

In [ ]:
input_dataset_id = widgets.Text(
 placeholder="<DATASET_ID>",
 description="Dataset ID:"
 )
input_description = widgets.Text(
 placeholder="<REPO_DESCRIPTION>",
 description="Description:"
 )
input_name = widgets.Text(
 placeholder="<NAME>",
 description="Name:"
 )
input_path = widgets.Text(
 placeholder="<PATH>",
 description="Path:"
 )
input_project_id= widgets.Text(
 placeholder="<GCP_PROJECT_ID>",
 description="GCP Project ID:",
 style=input_style
 )
display(cloningDropDown)
display(input_dataset_id)
display(input_description)
display(input_name)
display(input_path)
display(input_project_id)

# define a variable that will be reset when the button is pressed
buttonOutput = 'Please provide appropriate values in the input boxes.'

# define a function for the button to call
def button_click_event(b):
    with output:
        global buttonOutput
        
        terraCommand = """terra resource add-ref git-repo \\
        --cloning={0} \\
        --dataset_id={1} \\
        --description={2} \\
        --name={3} \\
        --path={4} \\
        --project_id={5} \\
        --workspace={6}
        """.format(cloningDropDown.value,input_dataset_id.value,input_description.value,input_name.value,input_path.value,input_project_id.value,WORKSPACE_ID)       
        print(terraCommand)
        os.system(terraCommand)

# get a reference to the widget output
output = widgets.Output()

button = widgets.Button(
    description='Add resource',
    disabled=False,
    button_style='',
    tooltip='abcd',
    icon='plus'
)

#bind the button_click_event to the button call event
button.on_click(button_click_event)

# show the current state of the output
print(buttonOutput)

#display the button
display(button, output)

#### 3. Add a BigQuery table

In [ ]:
input_dataset_id = widgets.Text(
 placeholder="<DATASET_ID>",
 description="Dataset ID:"
 )
input_description = widgets.Text(
 placeholder="<REPO_DESCRIPTION>",
 description="Description:"
 )
input_name = widgets.Text(
 placeholder="<NAME>",
 description="Name:"
 )
input_path = widgets.Text(
 placeholder="<PATH>",
 description="Path:"
 )
input_project_id= widgets.Text(
 placeholder="<GCP_PROJECT_ID>",
 description="GCP Project ID:",
 style=input_style
 )
input_table_id= widgets.Text(
 placeholder="<TABLE_ID>",
 description="Table ID:",
 style=input_style
 )
display(cloningDropDown)
display(input_dataset_id)
display(input_description)
display(input_name)
display(input_path)
display(input_project_id)
display(input_table_id)

# define a variable that will be reset when the button is pressed
buttonOutput = 'Please provide appropriate values in the input boxes.'

# define a function for the button to call
def button_click_event(b):
    with output:
        global buttonOutput
        
        terraCommand = """terra resource add-ref git-repo \\
        --cloning={0} \\
        --dataset_id={1} \\
        --description={2} \\
        --name={3} \\
        --path={4} \\
        --project_id={5} \\
        --table_id={6} \\
        --workspace={7}
        """.format(cloningDropDown.value,input_dataset_id.value,input_description.value,input_name.value,input_path.value,input_project_id.value,input_table_id.value,WORKSPACE_ID)       
        print(terraCommand)
        os.system(terraCommand)

# get a reference to the widget output
output = widgets.Output()

button = widgets.Button(
    description='Add resource',
    disabled=False,
    button_style='',
    tooltip='Click to add a BigQuery table as a referenced resource.',
    icon='plus'
)

#bind the button_click_event to the button call event
button.on_click(button_click_event)

# show the current state of the output
print(buttonOutput)

#display the button
display(button, output)

#### 4. Add a Google Cloud Storage bucket

#### 5. Add Google Cloud Storage object

## Provenance

Generate information about this notebook environment and the packages installed.

In [ ]:
!date

Conda and pip installed packages:

In [ ]:
!conda env export

JupyterLab extensions:

In [ ]:
!jupyter labextension list

Number of cores:

In [ ]:
!grep ^processor /proc/cpuinfo | wc -l

Memory:

In [ ]:
!grep "^MemTotal:" /proc/meminfo

---
Copyright 2022 Verily Life Sciences LLC

Use of this source code is governed by a BSD-style   
license that can be found in the LICENSE file or at   
https://developers.google.com/open-source/licenses/bsd